# **Classification de genres musicaux**

License : CC BY-NC-SA 4.0

L'objectif de ce TP est d'entraîner un modèle afin de classifier automatiquement une musique selon son genre musical.


Pour cela, nous allons faire usage d'une base de données (GTZAN) qui va servir à l'entrainement de plusieurs modèles. 

Nous utiliserons le framework PyTorch ainsi que ses extensions torchaudio/torchvision. 

Les calculs seront accélérés matériellement par l'usage des GPUs de Jean-Zay.


Ce notebook permet d'explorer et de se familiariser avec le dataset GTZAN.

> **Vérifions les modules chargés et la présence du ou des GPUs réservés : le notebook doit avoir été lancé depuis un nœud de calcul et non une frontale pour profiter de GPUs.**


> **La commande "!nvidia-smi" permet de rapidement visualiser l'état des GPUs.**

In [ ]:
!module list
!conda env list
!nvidia-smi
from torch.cuda import device_count
print('Torch detecting {0} GPUs compatible with CUDA'.format(device_count()))

## **Imports**

Plusieurs librairies seront utilisées tout au long du TP afin de procéder aux transformations des données, à la gestion de fichiers, à la visualisation, ...

> **Nous utilisons l'environnement pytorch-gpu/py3/1.11.0 qui comporte l'ensemble des librairies utiles pour ce TP.**

> **Vous pouvez modifier les scripts importés et les ré-importer sans relancer le kernel grâce à autoreload.**

In [ ]:
#!module load pytorch-gpu/py3/1.9.0
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np # Maths
import pandas as pd #Data analysis and manipulation

import os # Operating System interaction

import matplotlib # Visualization
import matplotlib.pyplot as plt
#Nécessaire pour visualiser les figures dans le notebook
%matplotlib inline 

from tqdm import tqdm #progress bar


import torch
from torch import optim
from torch import nn

> **Attention : sous windows, il faut installer soundfile (pip install soundfile) et ajouter la ligne suivante :**

> torchaudio.set_audio_backend("sox_io")

In [ ]:
# Fixe l'aléatoire pour la reproductibilité
np.random.seed(0)
torch.manual_seed(0)

# Quelques variables pour la gestion des fichiers
project_dir = os.getcwd()
data_dir = '/mnt/datasets/TP_Methodologie/GTZAN/data'
figures_dir = os.path.join(project_dir, 'figures')
scripts_dir = os.path.join(project_dir, 'scripts')
models_dir = os.path.join(project_dir, 'models')
dataset_dir = os.path.join(data_dir, 'raw')

---
---
---
---
---

# **Exploratory Data Analysis**

**Avant d'utiliser la base de données pour entraîner un modèle, il est important de procéder à une analyse de celle-ci. On parle d'EDA pour Exploratory Data Analysis. Les objectifs sont multiples :**
- comprendre les données
- nettoyer la base de données
- identifier des relations/dépendances

> **Prendre le temps de connaître les données est primordial. Il est possible d'anticiper plusieurs problèmes avant même d'entraîner le modèle**


**Plusieurs actions peuvent être ainsi prises pour :**
- retirer les données corrompues qui peuvent générer des bugs lors de leur utilisation
- supprimer les données redondantes qui alourdissent la base de données
- détecter des outliers qui peuvent nuire à l'entraînement
- identifier des déséquilibres entre classes et décider d'une stratégie pour les contrecarrer
- comprendre la complexité des données et du problème


Pour effectuer une EDA, il est souvent nécessaire d'utiliser une variété d'outils :
- de visualisation permettant de mieux interpréter les données
- statistiques qui permettent d'identifier quelques propriétés de la base
- de réduction de dimensionnalités pour compresser l'information, identifier des outliers, visualiser des relations entre classes par exemples




## **Le dataset**

Nous importons la base de données GTZAN (http://marsyas.info/downloads/datasets.html) en utilisant une version modifiée de la class proposée par Torchaudio.
Cela nous permet de profiter des corrections apportées à la version originale : suppression des doublons, partage de la base en sets pour l'entraînement, la validation et le test.
Une modification a été apportée afin de permettre l'application de transformations à la base de données lors du chargement.

De plus, la classe générée correspond aux standards des classes "Dataset" et "DataLoader" de Pytorch. Ceci permet de profiter efficacement des méthodes associées à ces classes.

In [ ]:
from scripts.dataset import GTZAN
from scripts.utils import list_classes
dataset = GTZAN(root=dataset_dir,download=True)
genres_dir = os.path.join(dataset_dir, 'genres')
classes = list_classes(genres_dir)
print(type(dataset))
print('Le dataset chargé comporte {} extraits musicaux différents.'.format(len(dataset)))
print('Il est divisé en {} genres ayant chacun 100 extraits.'.format(len(classes)))

> **Décommenter la ligne suivante pour écouter un des extrait si l'environnement le permet.**
> **Pour la suite de ce notebook, nous utiliserons le dataset d'entraînement corrigé par TorchAudio (suppression d'extraits identiques).** Cette version a déjà retiré les données redondantes ou endommagées de la base.
> Vous pouvez aussi changer "training" par "validation" ou "testing" pour en découvrir les tailles et éléments.

In [ ]:
dataset = GTZAN(root=dataset_dir, subset='training', download=True)
print('Le dataset chargé comporte {} extraits musicaux différents.'.format(len(dataset)))

> **Quelle est la taille totale du dataset filtré?**

## **Données audio**

Les extraits sont tous des fichiers audio Mono, 16 bits échantillionnés à 22050 Hz disponibles au format .wav.

Le dataset permet de récupérer pour chaque extrait la forme d'onde, son échantillonnage et son genre.

La forme d'onde désigne la variation de la pression de l'air (onde acoustique) au cours du temps.

In [ ]:
track = 0
waveform,samplerate,label,target= dataset[track]
print(" dimensions de l'oscillogramme {} \n échantillonnage : {} \n genre : {} ".format(waveform.size(),samplerate,label))

> **Décommenter la ligne suivante pour écouter un des extrait si l'environnement le permet.**

> **Pourquoi le taux d'échantillonnage est-il important?**

In [ ]:
import IPython
#IPython.display.Audio(waveform,rate=samplerate)

In [ ]:
from scripts.utils import check_sizes
sizes_uniformity, sizes = check_sizes(dataset)

Chaque signal correspond à un ensemble d'amplitude mesurées au fil du temps à la fréquence d'échantillonnage de 22050 Hz.

> **On peut d'ailleurs identifier que tous les enregistrements n'ont pas exactement la même durée ce qui peut poser des soucis pour certaines fonctions. On peut déjà anticiper certaines erreurs à venir et réfléchir aux possibles solutions.**

### **Visualisation**

Il est facilement possible de visualiser ce signal en affichant l'évolution de l'amplitude à chaque échantillon (temps).

In [ ]:
#import librosa.display
plt.figure(figsize=(13, 8))
#librosa.display.waveshow(waveform.numpy().reshape((-1,)), sr=samplerate)
plt.title(f"Forme d'onde de la musique (fréquence d'échantillonnage : {samplerate}Hz)")
plt.xlabel("échantillons")
plt.ylabel("amplitude")
plt.plot(waveform.t().numpy())

Pour classifier les extraits musicaux, il est tout à fait possible d'essayer d'utiliser le signal dans sa version originale.

Cependant, cela est généralement déconseillé. Un tel signal ne met pas efficacement en avant les caractéristiques utiles permettant d'identifier le genre.

Deux pistes que l'on peut croiser sont exploitables :
- trouver un modèle efficace pour notre type de données
- faire ressortir les caractéristiques de sorte à simplifier l'exercice pour le classifieur

---
---
---
---
---

## **Extraction des caractéristiques**

À partir de ce signal, il est possible :
- d'appliquer des transformations modifiant le signal
- d'extraire des informations du signal

Certaines transformations peuvent d'ailleurs être bénéfiques à la tâche que l'on souhaite accomplir. Par exemple, normaliser les amplitudes entre les extraits pour compenser des écarts d'intensité lors de la captation peut être bénéfique.



### **Des caractéristiques construites sur mesure**

Il existe une très grande variété de caractéristiques que l'on peut extraire d'un signal sonore. Cela va de concepts relativement simples tel que le nombre de passages par zéro de l'amplitude (zero crossing rate), en passant par des statistiques fréquentielles sur l'ensemble de l'extrait musical.

Beaucoup de caractéristiques reposent sur les transformées de Fourier. Celles-ci permettent de décomposer un signal dans l'ensemble des fréquences qui le compose. Il est commun d'appliquer d'utiliser des statistiques de ces caractéristiques pour avoir à la fin un descripteur compact d'informations choisies à l'aide de connaissances musicales dans l'objectif de classer nos musiques. On trouvera dedans des informations comme le tempo, les tonnetz, les harmoniques, ...

> Pour extraire ces caractéristiques, il existe plusieurs librairies dont Librosa : https://librosa.org/doc/latest/index.html dont vous pouvez consulter la documentation pour plus d'information sur chaque caractéristique. 

In [ ]:
features_path = os.path.join(data_dir, 'features', 'features_30_sec.csv')
data = pd.read_csv(features_path)
data.head()

> **Les fichiers feature features_30_sec.csv et features_3_sec.csv comportent les moyennes et variances pour chaque extrait des 30 caractéristiques suivantes :**

In [ ]:
for col in data.columns:
    print(col)

> Il est tout à fait possible de construire un descripteur à partir de caractéristiques et de l'utiliser pour la classification. La corrélation entre celles-ci peut aider à la sélection des caractéristiques.

> **Visualisez par exemple la corrélation entre les variances. Testez ensuite celle entre les moyennes. Pour cela, changez le paramètre stat dans la fonction compute_correlation_matrix.**

In [ ]:
# Computing the Correlation Matrix
from scripts.utils import compute_correlation_matrix
corr = compute_correlation_matrix(data, stat='var', show = True) # Choose stat between mean and var

> **On peut chercher à confirmer le choix de certaines caractéristiques en visualisant la distribution des classes pour une caractéristique. Utilisez le paramètre feature dans la fonction boxplot pour vérifier les classes que vous avez pu trouver intéressantes à partir des corrélations au-dessus.**

In [ ]:
from scripts.utils import boxplot
boxplot(data, feature='perceptr_var')

> Une sélection manuelle est généralement fastidieuse. Il existe des méthodes permettant de réduire la dimensionnalité des données tout en conservant le maximum d'information.

In [ ]:
from scripts.utils import pca
pca(data)

> **La PCA permet de visualiser quelles classes sont les plus similaires et d'anticiper ainsi les classes qui poseront des difficultés. On peut ainsi chercher à trouver une solution, possiblement des caractéristiques qui permettent de distinguer ces classes.**

## **Base de référence pour les performances du modèle**

Avec ces données, il est tout à fait possible d'entraîner un premier classifieur simple afin d'avoir une base de performance.

> **Par défauts, toutes les caractéristiques numériques sont utilisées, mais vous pouvez aussi en sélectionner par vous-même certaines.**

In [ ]:
from sklearn import preprocessing

features = data.loc[:, (data.columns != 'filename') * (data.columns != 'label')] #entrées : Ne pas sélectionner les noms de fichiers ou les labels
labels = data['label'] # sorties


cols_names = features.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)

# new data frame with the new scaled data. 
features = pd.DataFrame(np_scaled, columns = cols_names)
features.head()

Pour pouvoir évaluer notre modèle, il faut séparer le dataset en set d'entraînement et de test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=2021)

> **Une grande sélection d'algorithmes sont disponibles dans des librairies populaires telle que sklearn. Pour un jeu de données de cette taille avec un nombre de caractéristiques réduit, il est possible d'entraîner un certain nombre de modèles de Machine Learning assez rapidement pour comparer leurs performances. Vous trouverez à cette adresse les classifieurs disponibles avec sklearn : https://scikit-learn.org/stable/supervised_learning.html#supervised-learning . Vous ne devriez avoir qu'à changer la variable "model" et à importer la classe.**

In [ ]:
import sklearn

In [ ]:
sklearn.__version__

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
model = SVC()
model.fit(X_train, y_train)
preds = model.predict(X_test)
#print(confusion_matrix(y_test, preds))
print('Model Accuracy' ':', round(accuracy_score(y_test, preds), 5), '\n')

In [ ]:
# Confusion Matrix
import sklearn
from sklearn.metrics import confusion_matrix
import seaborn as sn


gtzan_genres = [
    "blues",
    "classical",
    "country",
    "disco",
    "hiphop",
    "jazz",
    "metal",
    "pop",
    "reggae",
    "rock",
]


cm = sklearn.metrics.confusion_matrix(y_test, preds, normalize='true') #normalize='true' 
df_cm = pd.DataFrame(cm, index = [i for i in gtzan_genres],
                  columns = [i for i in gtzan_genres])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

>**Testez d'entraîner un modèle en utilisant les caractéristiques sur des extraits de 3 secondes. Pour cela, modifiez la ligne :**

>**features_path = os.path.join(data_dir, 'features', 'features_30_sec.csv'** avec **features_3_sec.csv**

### **Des représentations plus généralistes**

Plutôt que de chercher à décrire l'extrait musical dans son ensemble, il est naturel de vouloir conserver l'aspect séquentiel de la musique.

Pour cela, une technique commune consiste à appliquer une représentation fréquentielle sur une fenêtre glissante plus petite que l'extrait musical pour représenter l'évolution des fréquences au cours du temps.
Il existe une grande variété de représentations qui applique des quantifications et échantillonnage différents ainsi que des mises à l'échelle différentes pour par exemple prendre en compte les différences de perception humaine de l'intensité selon la fréquence.

Ces représentations sont aussi disponibles depuis Librosa, nous utiliserons cependant Torchaudio pour les performances accrues et la compatibilité avec Pytorch et Torchvision (compatibilité avec le type Tensor).

> **Utilisez cette partie pour définir les transformations que vous souhaitez appliquer à la base de données. Vous devriez chercher à conserver l'aspect temporel de l'extrait audio, tout en compressant l'information de sorte à se rapprocher des données utilisées pour la conception des modèles LeNet et VGG (des images couleurs de taille 256*256).**

> Vous pouvez tester ci-dessous de modifier les transformations appliquées lors du chargement des données et visualiser celles-ci.

> Selon les caractéristiques sélectionnées et transmises aux modèles, les performances peuvent varier. Diverses caractéristiques et représentations peuvent être sélectionnées facilement avec torchaudio.transforms ou encore torchvision.transforms.
> Des transformations utiles sont disponibles avec https://pytorch.org/audio/0.11.0/transforms.html et https://pytorch.org/vision/0.12/transforms.html.


> Changer les caractéristiques, les modifier en les normalisant, ou appliquer diverses transformations affecte significativement les performances.


> torch.nn.Sequential et  torchvision.transforms.Compose permettent de combiner les transformations.

> **Utilisez **tvt.Lambda** pour intégrer des transformations ne provenant pas des modules torchaudio ou torch vision.**

> Attention toutes les entrées n'ont pas la même dimension. Toutes les sorties doivent avoir la même dimension.

> **crop_audio permet de mettre tous les extraits à la même durée. Vous aurez besoin de tvt.Lambda pour l'intégrer dans votre transformation.**

> Quelques fonctions utiles : 
> - torchvision.transforms.Lambda
> - torchvision.transforms.Crop
> - torchvision.transforms.Pad
> - torchvision.transforms.Resize
> - torchaudio.transforms.Spectrogram
> - torchaudio.transforms.AmplitudeToDB
> - torchaudio.transforms.MelScale
> - torchaudio.transforms.MelSpectrogram
> - torchaudio.transforms.MFCC

In [ ]:
from torch import nn
import torchaudio.transforms as tat
import torchvision.transforms as tvt
import torchvision
from scripts.utils import crop_audio
#from torchvision import transforms, utils
#from torchvision.transforms import ToTensor, Lambda, Resize
#Crop = tvt.Lambda(lambda waveform: crop_audio(waveform, duration = 20, augment=False))
transformation = tvt.Compose([tat.Spectrogram(), tvt.Resize(size=[201, 3306])])
dataset = GTZAN(root=dataset_dir, download=True, transform=transformation)

In [ ]:
from scripts.utils import check_sizes
sizes_uniformity, sizes = check_sizes(dataset)

In [ ]:
features = dataset[0][0].log2().numpy().squeeze()
plt.figure(figsize=(30, 8))
plt.title("Spectrogramme de l'extrait musical")
plt.imshow(features,cmap='inferno') 
plt.colorbar()

> Si vous êtes daltonien, essayez une autre cmap  : https://matplotlib.org/stable/tutorials/colors/colormaps.html


A noter : les transformations sont appliquées à l'appel de l'élément dans la base de données et non lors de la définition de celle-ci.

---
---
---
---
---

# Pour libérer une partie de la mémoire, vous pouvez rédemarrer le kernel, en particulier si vous passez au notebook suivant.